In [5]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym
from scipy.integrate import solve_ivp
from scipy import integrate
import jax
import jax.numpy as jnp
import cvxpy as cp
from jax.typing import ArrayLike
from jax import jit
import random as rand
import plotly.graph_objects as go
import time 

## Dubin Model testbed: 
Planned tests:
1. Sequential convex Optmization
2. Sucessive Convexification
3. MPC + convex optmization 

In [6]:
import matplotlib.pyplot as plt
import numpy as np

def plotTrajectory(trajectory, goal=None, title="Trajectory Plot", arrow_step=10):
    """
    Plots a given trajectory of states with arrows indicating the orientation.
    
    Parameters:
    - trajectory: List or array of states [x, y, theta] over time.
    - goal: Optional tuple (x_goal, y_goal) to mark the goal position on the plot.
    - title: Title of the plot.
    - arrow_step: Number of steps between arrows (default is 10).
    """
    # Extract x, y, and theta
    x = trajectory[:, 0]
    y = trajectory[:, 1]
    theta = trajectory[:, 2]

    # Plot the trajectory
    plt.figure(figsize=(5, 5))
    

    # Add arrows to show orientation
    for i in range(0, len(x), arrow_step):
        dx = np.cos(theta[i]) * 0.5  # Scale the arrow length
        dy = np.sin(theta[i]) * 0.5
        plt.arrow(x[i], y[i], dx, dy, head_width=0.2, head_length=0.3, color="blue")

    # Mark the start position
    plt.scatter(x[0], y[0], color="green", label="Start")

    # Mark the goal position if provided
    if goal is not None:
        x_goal, y_goal = goal
        plt.scatter(x_goal, y_goal, color="red", label="Goal")

    # Plot settings
    plt.xlabel("X")
    plt.xlim(-5, 5)
    plt.ylim(-5, 5)
    plt.plot(x, y, label="Trajectory", linewidth=2)
    plt.ylabel("Y")
    plt.title(title)
    #plt.axis("equal")
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
def dynamics(state, control):
    x, y, theta = state
    v, w = control

    x_dot = v*jnp.cos(theta)
    y_dot = v*jnp.sin(theta)
    theta_dot = w

    state_dot = jnp.array([x_dot, y_dot, theta_dot])
   
    return state_dot


# linearization of dynamics
def linearize_autodiff3( state, control):

    func_x = jit(jax.jacfwd(dynamics, argnums=0))
    func_u = jit(jax.jacfwd(dynamics, argnums=1))

    A = func_x(state, control)
    B = func_u(state, control)

    return A, B




def controlSimple(state, goalstate, kv, kw):
    x, y, theta = state
    x_goal, y_goal, theta_goal = goalstate

    

    d = jnp.sqrt((x_goal-x)**2+(y_goal-y)**2)
    v = kv * d

    theta_now= np.arctan2(y_goal-y, x_goal-x)
    theta_now = theta_now % (2*np.pi)
    print(np.rad2deg(theta_now))
    theta_err = theta_now - theta
    theta_err = (theta_err + np.pi) % 2*np.pi - np.pi  # Wrap to [-180, 180] for shortest path

    #print(np.rad2deg(theta_err))
    w = -kw * theta_err
 

    control = [v, w]
    return control






# ode solver
def trajgGenBasic(init_state, goal_state, kv, kw, timerun, dt):
    traj =[init_state]
    controls = []
    for i in range(int(timerun/dt)):
       
        control = controlSimple(traj[i], goal_state, kv, kw)
       
        state = dynamics(traj[i], control) *dt + traj[i]
        


        if (i%10 == 0):
            print(i)
            print("control = ", control)
            print("state = ", state)
        
        traj.append(state)
        controls.append(control)
    return traj, controls

# ode solver -> for loop
def trajGenBasicLinearized(init_state, goal_state, kv, kw, trust_region):
    traj =[]

    return traj

def doubleintegratormodel(init_state, goal_state, kv, kw, timerun, dt):
    traj =[init_state]
    controls = []
    for i in range(int(timerun/dt)):
       
        control = controlSimple(traj[i], goal_state, kv, kw)
       
        u, w = control
        A = jnp.array([[0, 1], [0,0]])
        B = jnp.array([[0,1]])

        state = A@(traj[i].T) + B@(control.T)
        
        if (i%10 == 0):
            print(i)
            print("control = ", control)
            print("state = ", state)
        
        traj.append(state)
        controls.append(control)
    return traj, controls

    



In [8]:

angle_init = -180
init_state = jnp.array([1, 0, jnp.deg2rad(angle_init)], dtype=jnp.float32)
goal_state = jnp.array([0, 0, 0], dtype=jnp.float32)
print(init_state)
init_control = jnp.array([0, 0],dtype=jnp.float32)

A, B = linearize_autodiff3(init_state, init_control)
print("A:", A)
print("B:",B)


kv = .1
kw = 0
timerun = 30
dt = .1


#traj, control = trajgGenBasic(init_state, goal_state, kv, kw, timerun, dt)

traj = doubleintegratormodel(init_state, goal_state, kv, kw, timerun, dt)


traj = jnp.vstack(traj)
plotTrajectory(traj, goal=None, title="Trajectory Plot", arrow_step=20)


[ 1.         0.        -3.1415927]
A: [[-0. -0. -0.]
 [-0. -0. -0.]
 [ 0.  0.  0.]]
B: [[-1.000000e+00 -0.000000e+00]
 [ 8.742278e-08  0.000000e+00]
 [ 0.000000e+00  1.000000e+00]]
180.00000500895632


TypeError: dot_general requires contracting dimensions to have the same shape, got (2,) and (3,).